In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Import Libraries
import os
import cv2
import random
import numpy as np
from PIL import Image, ImageEnhance
from tqdm import tqdm
from pathlib import Path

# Step 3: Set Paths
input_dir = Path("/content/drive/MyDrive/test/resize")
output_dir = Path("/content/drive/MyDrive/test/bri")
output_dir.mkdir(parents=True, exist_ok=True)

# Step 4: Define Augmentation Functions
def flip_image(image): return cv2.flip(image, 1)

def rotate_image(image):
    angle = random.randint(-30, 30)
    h, w = image.shape[:2]
    matrix = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    return cv2.warpAffine(image, matrix, (w, h))

def scale_image(image):
    scale_factor = random.uniform(0.8, 1.2)
    return cv2.resize(image, None, fx=scale_factor, fy=scale_factor)

def translate_image(image):
    tx, ty = random.randint(-20, 20), random.randint(-20, 20)
    matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]))

def crop_image(image):
    h, w = image.shape[:2]
    start_x = random.randint(0, w // 10)
    start_y = random.randint(0, h // 10)
    end_x = w - random.randint(0, w // 10)
    end_y = h - random.randint(0, h // 10)
    return image[start_y:end_y, start_x:end_x]

def add_noise(image):
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
    return cv2.add(image, noise)

def adjust_brightness(image):
    enhancer = ImageEnhance.Brightness(Image.fromarray(image))
    return np.array(enhancer.enhance(random.uniform(0.7, 1.3)))

def adjust_contrast(image):
    enhancer = ImageEnhance.Contrast(Image.fromarray(image))
    return np.array(enhancer.enhance(random.uniform(0.7, 1.3)))

def cutout(image, mask_size=50):
    h, w = image.shape[:2]
    y, x = np.random.randint(h), np.random.randint(w)
    y1 = np.clip(y - mask_size // 2, 0, h)
    y2 = np.clip(y + mask_size // 2, 0, h)
    x1 = np.clip(x - mask_size // 2, 0, w)
    x2 = np.clip(x + mask_size // 2, 0, w)
    image[y1:y2, x1:x2] = 0
    return image

def gridmask(image, grid_size=100, mask_ratio=0.5):
    h, w = image.shape[:2]
    for y in range(0, h, grid_size):
        for x in range(0, w, grid_size):
            if random.random() < mask_ratio:
                image[y:y+grid_size//2, x:x+grid_size//2] = 0
    return image

def motion_blur(image, kernel_size=15):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel /= kernel_size
    return cv2.filter2D(image, -1, kernel)

def color_jitter(image):
    image = Image.fromarray(image)
    image = ImageEnhance.Color(image).enhance(random.uniform(0.7, 1.3))
    image = ImageEnhance.Brightness(image).enhance(random.uniform(0.7, 1.3))
    image = ImageEnhance.Contrast(image).enhance(random.uniform(0.7, 1.3))
    return np.array(image)

def emboss(image):
    kernel = np.array([[ -2, -1, 0],
                       [ -1,  1, 1],
                       [  0,  1, 2]])
    return cv2.filter2D(image, -1, kernel)

def equalize(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def posterize(image, bits=4):
    shift = 8 - bits
    return np.right_shift(np.left_shift(image, shift), shift)

def fog_effect(image, fog_factor=0.5):
    overlay = np.full_like(image, 255)
    return cv2.addWeighted(image, 1 - fog_factor, overlay, fog_factor, 0)

def rain_effect(image, drop_length=20, drop_width=1, drop_color=(200, 200, 200), drop_count=100):
    rain = np.copy(image)
    h, w = rain.shape[:2]
    for _ in range(drop_count):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h - drop_length)
        cv2.line(rain, (x, y), (x, y + drop_length), drop_color, drop_width)
    return cv2.blur(rain, (3, 3))

def mixup(image1, image2):
    lam = np.random.beta(0.4, 0.4)
    image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))
    return (lam * image1 + (1 - lam) * image2).astype(np.uint8)

# Step 5: Load Images (Recursively)
image_files = [f for f in input_dir.rglob("*") if f.suffix.lower() in [".jpg", ".jpeg", ".png"]]

if not image_files:
    raise FileNotFoundError(f" No images found in {input_dir}. Please check the folder or image file extensions.")

print(f"Found {len(image_files)} images for augmentation.")

# Set your augmentation target count here
target_count = 20
current_count = len(image_files)
augmentation_needed = max(0, target_count - current_count)

augmentations = [
    flip_image, rotate_image, scale_image, translate_image,
    crop_image, add_noise, adjust_brightness, adjust_contrast,
    cutout, gridmask, motion_blur, color_jitter, emboss,
    equalize, posterize, fog_effect, rain_effect
]

# Step 6: Augment and Save
for i in tqdm(range(augmentation_needed)):
    img_path = random.choice(image_files)
    img = cv2.imread(str(img_path))

    if random.random() < 0.1:
        img2_path = random.choice(image_files)
        img2 = cv2.imread(str(img2_path))
        aug_img = mixup(img, img2)
    else:
        aug_fn = random.choice(augmentations)
        aug_img = aug_fn(img)

    aug_img = cv2.resize(aug_img, (800, 800))
    pil_img = Image.fromarray(cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB))
    save_path = output_dir / f"aug_{i}.jpg"

    quality = 95
    while True:
        pil_img.save(save_path, "JPEG", quality=quality, dpi=(120, 120))
        if os.path.getsize(save_path) <= 100 * 1024 or quality <= 60:
            break
        quality -= 5

print("Augmentation complete!")
